In [1]:
from lime import lime_image
import warnings
warnings.filterwarnings("ignore")

import os
import shap
import numpy as np
np.object = object
np.int = int
np.bool = bool
import pandas as pd
import tensorflow as tf
from typing import List
from lime import lime_image
from tensorflow import keras
import ipywidgets as widgets
import matplotlib.pyplot as plt
from IPython.display import display
from tf_keras_vis.utils import num_of_gpus
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tf_keras_vis.utils.model_modifiers import ReplaceToLinear

import zipfile



In [2]:

from test_utils import test_scores
from modeling_utils import get_model
from mics_utils import load_config, update_config_file
from dataset_utils import create_df, create_model_data, show_images
from encrypt_utils import encrypt_image, decrypt_image, plot_images
from federated_learning_utils import Client, EdgeServer, GlobalServer
from plotting_utils import  plot_training, plot_dataset_split_distribution, plot_dataset_classwise_distributions
from training_utils import train_our_federated_learninig_model, train_traditional_federated_learninig_model, train_individual_clients

from xai_utils import (
    display_images_in_grid,
    get_images_for_xai_binary,
    get_images_for_xai_multi,
    calculate_infidelity,
    calculate_sensitivity,
    predict_and_display,
    create_shap_explanation,
    create_heatmap_and_superimpose,
    create_saliency_model,
    create_gradcam_model,
    create_gradcam_plus_plus_model,
    create_scorecam_model,
    create_faster_scorecam_model,
    create_lime_explanation,
    combine_heatmaps,
    plot_xia_interpretability
    )


Install required packages

Imports

Set the parameters

In [3]:
# Overall parameters
epochs = 1
batch_size = 10
data_split_ratio = 0.1

# Federated learning parameters
iterations = 1
num_clients = 4
num_edge_servers = 2
local_model_epochs = 3
to_evaluate_client_model_index = 0

# XAI parameters
# Specify the desired layer for GradCAM
layer_id = -1
layer_name = 'top_conv'  # The last convolutional layer in EfficientNetB3
backbone_model_layer_name = 'efficientnetb3'

# Plotting params
plot_distributions = False
plot_sampled_images = False


# Number of GPUs
_, gpus = num_of_gpus()
print('Tensorflow recognized {} GPUs'.format(gpus))

Tensorflow recognized 0 GPUs


In [4]:
dataset_dropdown_options = ['Lukemia', 'Oral', 'Lymphoma', 'HIV']
dataset_dropdown = widgets.Dropdown(options=dataset_dropdown_options, description='Select a dataset:')
display(dataset_dropdown)

technique_dropdown_options = ['Simple', 'Traditional_Federated_Learning', 'Our_Federated_Learning', 'Individual_Clients']
technique_dropdown = widgets.Dropdown(options=technique_dropdown_options, description='Select a model training technique:')
display(technique_dropdown)

Dropdown(description='Select a dataset:', options=('Lukemia', 'Oral', 'Lymphoma', 'HIV'), value='Lukemia')

Dropdown(description='Select a model training technique:', options=('Simple', 'Traditional_Federated_Learning'…

Dataset

In [5]:
datasets = []
channels: int = 0
data_dir: str = ""
dataset_type: int = 0
dataset_path: str = ""
number_of_classes: int = 0
class_labels: List[str] = []

selected_dataset = dataset_dropdown.value
selected_technique = technique_dropdown.value

if selected_dataset == 'Lukemia':

    dataset_type = 1
    number_of_classes = 2
    class_labels = ['all','hem']
    data_dir = 'D:/projects/lukemia/content/datasets/C-NMC_Leukemia/training_data'
    dataset_path = 'D:/projects/lukemia/content/drive/MyDrive/FL_Project/data/C-NMC_Leukemia.zip'
    update_config_file(selected_dataset,number_of_classes)

elif selected_dataset == 'Oral' or selected_dataset == 'Lymphoma':

    dataset_type = 2
    dataset_path = 'D:/projects/lukemia/content/drive/MyDrive/FL_Project/data/multi-cancer.zip'
    if selected_dataset == 'Oral':
        number_of_classes = 2
        class_labels = ['normal','infected']
        data_dir = 'D:/projects/lukemia/content/datasets/Multi Cancer/Oral Cancer'
        update_config_file(selected_dataset,number_of_classes)

    elif selected_dataset == 'Lymphoma':
        number_of_classes = 3
        class_labels = ['lymph_cll','lymph_fl','lymph_mfl']
        data_dir = 'D:/projects/lukemia/content/datasets/Multi Cancer/Lymphoma'
        update_config_file(selected_dataset,number_of_classes)

if (not os.path.exists(data_dir)) and dataset_path.endswith('.zip'):
    print('Dataset is a compressed file. Uncompressing:')
    #!unzip "{dataset_path}" -d D:/projects/lukemia/content/datasets
    if not os.path.exists("D:/projects/lukemia/content/datasets"):
        os.makedirs("D:/projects/lukemia/content/datasets")

    with zipfile.ZipFile(dataset_path, 'r') as zip_ref:
        zip_ref.extractall("D:/projects/lukemia/content/datasets")

elif not os.path.exists(data_dir):
    raise IOError('Dataset directory does not exist: {}'.format(data_dir))


print('Selected dataset: ', selected_dataset)
print('Model will be trained using selected technique: ', selected_technique)
global_train_df_path = f'D:/projects/lukemia/content/drive/MyDrive/FL_Project/data/{selected_dataset}_global_train.csv'
global_test_df_path = f'D:/projects/lukemia/content/drive/MyDrive/FL_Project/data/{selected_dataset}_global_test.csv'
prefix_path = r"D:/projects/lukemia"
print(global_train_df_path)

if (not os.path.exists(global_train_df_path)) or (not os.path.exists(global_test_df_path)):
    print('Creating standard training and test splits of data on global level.')
    global_train_df, valid_df, test_df = create_df(data_dir, dataset_type)
    global_test_df = pd.concat([valid_df, test_df])
    global_train_df.to_csv(global_train_df_path, index=None)
    global_test_df.to_csv(global_test_df_path, index=None)


global_train_df = pd.read_csv(global_train_df_path)
global_test_df = pd.read_csv(global_test_df_path)

global_train_df ["filepaths"] = prefix_path + global_train_df ["filepaths"]
global_test_df ["filepaths"] = prefix_path + global_test_df ["filepaths"]

datasets = []
all_client_local_test_data = pd.DataFrame()

if selected_technique == 'Simple':
    train_df, valid_df = train_test_split(global_train_df, test_size=data_split_ratio)
    test_df = global_test_df.copy(deep=True)
    all_client_local_test_data = global_test_df.copy(deep=True)
    train_gen, valid_gen, test_gen = create_model_data(train_df, valid_df, test_df, batch_size)
    datasets.append((train_gen, valid_gen, test_gen))
    print(f"Total train: {len(train_df)}, val: {len(valid_df)}, test: {len(test_df)}")
    if plot_distributions:
        print()
        plot_dataset_split_distribution(train_df, valid_df, test_df)
        print()
        plot_dataset_classwise_distributions(train_df, valid_df, test_df)
    if plot_sampled_images:
        print()
        show_images(test_gen)

else:
    for i in range(num_clients):
        print('For client',i+1)
        cl_df = global_train_df[(i*(len(global_train_df)//num_clients)):((i+1)*(len(global_train_df)//num_clients))]
        cl_train_df, cl_test_df = train_test_split(cl_df, test_size=data_split_ratio)
        all_client_local_test_data = pd.concat([all_client_local_test_data, cl_test_df])
        cl_train_df, cl_valid_df = train_test_split(cl_train_df, test_size=data_split_ratio)

        train_gen, valid_gen, test_gen = create_model_data(cl_train_df, cl_valid_df, cl_test_df, batch_size)
        datasets.append((train_gen, valid_gen, test_gen))
        print(f"Total train: {len(cl_train_df)}, val: {len(cl_valid_df)}, test: {len(cl_test_df)}")
        if plot_distributions:
            print()
            plot_dataset_split_distribution(cl_train_df, cl_valid_df, cl_test_df)
            print()
            plot_dataset_classwise_distributions(cl_train_df, cl_valid_df, cl_test_df)
        if plot_sampled_images:
            print()
            show_images(test_gen)

_, _, global_test_gen = create_model_data(global_train_df,global_train_df,global_test_df,batch_size)
_, _, all_client_local_test_gen = create_model_data(global_train_df,global_train_df,all_client_local_test_data,batch_size)

Selected dataset:  Lukemia
Model will be trained using selected technique:  Simple
D:/projects/lukemia/content/drive/MyDrive/FL_Project/data/Lukemia_global_train.csv
Found 8634 validated image filenames belonging to 2 classes.
Found 960 validated image filenames belonging to 2 classes.
Found 1067 validated image filenames belonging to 2 classes.
Total train: 8634, val: 960, test: 1067
Found 9594 validated image filenames belonging to 2 classes.
Found 9594 validated image filenames belonging to 2 classes.
Found 1067 validated image filenames belonging to 2 classes.
Found 9594 validated image filenames belonging to 2 classes.
Found 9594 validated image filenames belonging to 2 classes.
Found 1067 validated image filenames belonging to 2 classes.


Model Training

In [ ]:
model = get_model()
print(model.summary())

clients = []
client_models = []
confusion_matrix = []
config = load_config(file_path='D:/projects/lukemia/content/drive/MyDrive/FL_Project/conf/conf.yaml')
gpu_available = tf.config.list_physical_devices('GPU')
if gpu_available:
    print("GPU is available for training.")
else:
    print("GPU is not available. Training will be done on CPU.")

checkpoint_filepath = f'/content/drive/MyDrive/FL_Project/models/{selected_dataset}/{selected_technique}/model_checkpoint.h5'


if selected_technique == 'Simple':

    model = get_model()
    early_stopping = EarlyStopping(monitor='val_loss', patience=config['training_params']['patience'], verbose=1, restore_best_weights=True)
    model_checkpoint = ModelCheckpoint(checkpoint_filepath, monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)
    callbacks = [early_stopping, model_checkpoint]
    train_gen, valid_gen, test_gen = datasets[0]
    history = model.fit(x= train_gen, epochs= epochs, verbose = 1, callbacks= callbacks, validation_data= valid_gen)


elif selected_technique == 'Our_Federated_Learning':

    clients = [Client(c+1, datasets[c], f'{checkpoint_filepath}/client_{c+1}_new_model', local_model_epochs) for c in range(num_clients)]
    edge_servers = [EdgeServer(e+1) for e in range(num_edge_servers)]
    global_server = GlobalServer()

    if os.path.exists(f"{checkpoint_filepath}/fl_global_model"):
        global_server.global_model.load_weights(f"{checkpoint_filepath}/fl_global_model")

    train_our_federated_learninig_model(iterations, clients, edge_servers, global_server, checkpoint_filepath, global_test_gen)

    for client in clients:
        print(f'Evaluating client {client.id} on local test data.\n')
        _ = test_scores(client.test_gen, client.model)

        print(f'Evaluating client {client.id} on local global test data.\n')
        _ = test_scores(global_test_gen, client.model)

    model = global_server.global_model



elif selected_technique == 'Traditional_Federated_Learning':

    global_model = get_model()

    if os.path.exists(f"{checkpoint_filepath}/fl_global_model"):
        global_model.load_weights(f"{checkpoint_filepath}/fl_global_model")

    for i in range(num_clients):
        model = get_model()
        saved_model_path = f"{checkpoint_filepath}/{i}"
        if os.path.exists(saved_model_path):
            print('Loading saved model checkpoint!')
            model.load_weights()
        client_models.append(model)

    train_traditional_federated_learninig_model(iterations, datasets, client_models, global_model, checkpoint_filepath, epochs, global_test_gen)

    for i in range(num_clients):
        print(f'Evaluating client {i+1} on local test data.\n')
        cl_model = client_models[i]
        _, _, cl_test_gen = datasets[i]
        _ = test_scores(cl_test_gen, cl_model)
        print(f'Evaluating client {i+1} on global test data.\n')
        _ = test_scores(global_test_gen, cl_model)

    model = global_model


elif selected_technique == 'Individual_Clients':

    for i in range(num_clients):
        model = get_model()
        saved_model_path = f"{checkpoint_filepath}/{i}"
        if os.path.exists(saved_model_path):
            print('Loading saved model checkpoint!')
            model.load_weights()
        client_models.append(model)

    train_individual_clients(datasets, client_models, epochs, checkpoint_filepath)
    model = client_models[to_evaluate_client_model_index-1]

    for i in range(num_clients):
      print(f'Evaluating client {i+1} on global test data.\n')
      _ = test_scores(global_test_gen, client_models[i])


print('Evaluating global model on global test data.\n')
confusion_matrix = test_scores(global_test_gen, model)




Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb3 (Functional  (None, 1536)              10783535  
 )                                                               
                                                                 
 batch_normalization (Batch  (None, 1536)              6144      
 Normalization)                                                  
                                                                 
 dense (Dense)               (None, 256)               393472    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 2)                 514       
                                                                 
Total params: 11183665 (42.66 MB)
Trainable params: 1

Test of other clients data

In [ ]:
eval_mode_dropdown_options = ['All', 'Other']
eval_mode_dropdown = widgets.Dropdown(options=eval_mode_dropdown_options, description='Select a mode of evaluation:')
display(eval_mode_dropdown)

In [ ]:
if eval_mode_dropdown.value == 'Other':

    model_client_dropdown = widgets.Dropdown(options=list(range(1,num_clients+1,1)), description='Select model client ID:')
    display(model_client_dropdown)

    data_cleint_dropdown = widgets.Dropdown(options=list(range(1,num_clients+1,1)), description='Select data client ID:')
    display(data_cleint_dropdown)

else:
    model_client_dropdown = widgets.Dropdown(options=list(range(1,num_clients+1,1)), description='Select model client ID:')
    data_cleint_dropdown = widgets.Dropdown(options=list(range(1,num_clients+1,1)), description='Select data client ID:')
    display(model_client_dropdown)

In [ ]:
to_evaluate_model = None
to_evaluate_data = None
mode = eval_mode_dropdown.value
to_evaluate_model_client = int(model_client_dropdown.value) - 1
to_evaluate_data_client = int(data_cleint_dropdown.value) - 1


if selected_technique in ['Traditional_Federated_Learning', 'Our_Federated_Learning','Individual_Clients']:
    if selected_technique in ['Traditional_Federated_Learning','Individual_Clients']:
        to_evaluate_model = client_models[int(to_evaluate_model_client)]
    else:
        to_evaluate_model = clients[to_evaluate_model_client].model

    if mode == 'Other':
        if selected_technique in ['Traditional_Federated_Learning','Individual_Clients']:
            to_evaluate_data = datasets[to_evaluate_data_client][2]
        else:
            to_evaluate_data = clients[to_evaluate_data_client].test_gen
    else:
        if selected_technique in ['Traditional_Federated_Learning','Individual_Clients']:
            to_evaluate_data = all_client_local_test_gen

    if mode== 'Other':
        print(f"Evaluating local model of client {to_evaluate_model_client} on data of client {to_evaluate_data_client}.\n")
    else:
        print(f"Evaluating local model of client {to_evaluate_model_client} on test data of all local clients.\n")

    _ = test_scores(to_evaluate_data, to_evaluate_model)

Test Specific Client Model

In [ ]:
# # You can use following function to evaluate any model on any data, just replace model and data
# # you can also load saved model
# # Following can be used to load a client model and evaluate it for client given data
# model = get_model()
# model.load_weights(f"client/model/path")
# confusion_matrix = test_scores(global_test_gen, model)

Encryption/Decryption

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets

# Function to update and plot images
def update_and_plot(seed):
    np.random.seed(seed)
    key = np.random.randint(0, 256, size=image_shape, dtype=np.uint8)
    encrypted_image = encrypt_image(original_image, key)
    decrypted_image = decrypt_image(encrypted_image, key)
    plot_images(original_image, encrypted_image, decrypted_image)

# Load the original image
original_image = next(global_test_gen)[0][0] / 255
image_shape = original_image.shape

# Create a slider for seed value
seed_slider = widgets.IntSlider(value=0, min=0, max=1000, step=1, description='Seed:')

# Display the slider and bind the update function
interact(update_and_plot, seed=seed_slider)


In [ ]:

replace2linear = ReplaceToLinear()
lime_explainer = lime_image.LimeImageExplainer()
efficientnet_layer = model.get_layer(backbone_model_layer_name)
efficientnet_model = keras.models.Model(inputs=efficientnet_layer.input, outputs=efficientnet_layer.output)

shap.initjs()
method = "inpaint_telea"  # "inpaint_ns" or "inpaint_telea" or "blur(28,28)"
masker = shap.maskers.Image(method, image_shape)
shap_explainer = shap.Explainer(model, masker, output_names=class_labels)


true_pos_images, true_neg_images, false_pos_images, false_neg_images = [], [], [], []
if number_of_classes == 2:
    true_pos_images, true_neg_images, false_pos_images, false_neg_images = get_images_for_xai_binary(datasets, model, confusion_matrix)
    print("Total TP, TN, FP and FN images respectively: \n", len(true_pos_images), len(true_neg_images), len(false_pos_images), len(false_neg_images))

elif number_of_classes > 2:
    true_pos_images, true_neg_images, false_pos_images, false_neg_images = get_images_for_xai_multi(datasets, model, number_of_classes, confusion_matrix)
    print("Total TP, TN, FP and FN images respectively: \n", len(true_pos_images), len(true_neg_images), len(false_pos_images), len(false_neg_images))

import plotly.graph_objects as go
from plotly.subplots import make_subplots

def display_images_in_grid(images, title):
    # Determine the grid size for plotting (for simplicity, using square layout)
    grid_size = int(len(images)**0.5)
    fig = make_subplots(rows=grid_size, cols=grid_size, subplot_titles=[f"Image {i+1}" for i in range(len(images))])

    # Add each image to the subplot
    for i, image in enumerate(images):
        row = (i // grid_size) + 1
        col = (i % grid_size) + 1
        fig.add_trace(go.Image(z=image), row=row, col=col)

    # Update layout
    fig.update_layout(height=200 * grid_size, width=200 * grid_size, title_text=title)

    fig.show()

# Usage with your existing code
display_images_in_grid(true_pos_images, 'True Positive Images')
display_images_in_grid(false_pos_images, 'False Positive Images')
display_images_in_grid(true_neg_images, 'True Negative Images')
display_images_in_grid(false_neg_images, 'False Negative Images')


In [ ]:
import numpy as np
import matplotlib
matplotlib.use('Agg') 
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from skimage.segmentation import mark_boundaries
import torch

# Assuming the required libraries and models (SHAP, LIME, etc.) are already imported and initialized

def generate_explanations(image, model, shap_explainer, lime_explainer, title,pdf):
    # SHAP Explanation
    shap_values = shap_explainer(image[np.newaxis, ...])
    shap.image_plot(shap_values, show=False)
    plt.title(f"{title} - SHAP Explanation")
    pdf.savefig(plt.gcf())  # Save the figure
    plt.close()   
    #plt.show()

    # LIME Explanation
    explanation = lime_explainer.explain_instance(image, model.predict, top_labels=1, hide_color=0, num_samples=100)

    temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=False, num_features=10, hide_rest=False)
    plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))
    plt.title(f"{title} - LIME Explanation")
    #plt.show()
    pdf.savefig(plt.gcf())
    plt.close()
'''
# Function to display images in a grid
def display_images_in_grid(images, title,pdf):
    grid_size = int(len(images)**0.5)
    fig = make_subplots(rows=grid_size, cols=grid_size, subplot_titles=[f"Image {i+1}" for i in range(len(images))])

    for i, image in enumerate(images):
        row = (i // grid_size) + 1
        col = (i % grid_size) + 1
        fig.add_trace(go.Image(z=image), row=row, col=col)

    fig.update_layout(height=300 * grid_size, width=300 * grid_size, title_text=title)

    temp_png = "temp_plotly_figure.png"
    fig.write_image(temp_png)
    plotly_image = plt.imread(temp_png)
    plt.figure()
    plt.imshow(plotly_image)
    plt.axis('off')
    plt.title(title)
    
    # Save to PDF
    pdf.savefig(plt.gcf())
    plt.close()
'''

def display_images_in_grid(images, title, pdf):
    n_images = len(images)
    grid_size = int(np.ceil(np.sqrt(n_images)))
    
    fig, axes = plt.subplots(
        nrows=grid_size, 
        ncols=grid_size, 
        figsize=(4*grid_size, 4*grid_size)
    )
    axes = axes.ravel()

    for idx, ax in enumerate(axes):
        if idx < n_images:
            ax.imshow(images[idx], cmap='gray')
            ax.set_title(f"Image {idx+1}")
        ax.axis('off')
    
    plt.suptitle(title)
    pdf.savefig(fig)
    plt.close(fig)

with PdfPages(f"{selected_dataset}_explanations_output.pdf") as pdf:
    # Loop over each category and its images
    # Applying the function to each image category
    for category, images in [("True Positive", true_pos_images), ("True Negative", true_neg_images),
                             ("False Positive", false_pos_images), ("False Negative", false_neg_images)]:
        display_images_in_grid(images, f'{category} Images',pdf)
        for i, image in enumerate(images):
            generate_explanations(image, model, shap_explainer, lime_explainer, f"{category} Image {i+1}",pdf)
            if i >= 5:  # Limit to first 5 images per category
                break

In [ ]:

from tqdm import tqdm
def process_and_display_images(image_category, images):
    print(f'{image_category} Images')
    for i in tqdm(range(len(images)), desc=image_category):
        combined_heatmap = plot_xia_interpretability(images[i:i+1], model, efficientnet_model, replace2linear, shap_explainer, lime_explainer, class_labels, layer_name, layer_id)
        predict_and_display(images[i], model, class_labels, True, combined_heatmap)

# Process and display images for each category
process_and_display_images('True Positive', true_pos_images)
process_and_display_images('True Negative', true_neg_images)
process_and_display_images('False Positive', false_pos_images)
process_and_display_images('False Negative', false_neg_images)

In [ ]:
for img_types in ['true_pos_images', 'false_pos_images', 'true_neg_images', 'false_neg_images']:

  print(img_types)

  if img_types == 'true_pos_images':
    test_labels = np.array([0])
  elif img_types == 'false_pos_images':
    test_labels = np.array([0])
  elif img_types == 'true_neg_images':
    test_labels = np.array([1])
  elif img_types == 'false_neg_images':
    test_labels = np.array([1])

  test_images = true_neg_images

  for i in range(len(test_images)):
    test_image = test_images[i:i+1]

    infidelity = calculate_infidelity(model, model, test_image)
    display(f'Infidelity: {infidelity}')

    sensitivity = calculate_sensitivity(model, test_image[0])
    display(f'Sensitivity: {sensitivity}')

In [ ]:
import numpy as np
import tensorflow as tf
%matplotlib inline
%config InlineBackend.figure_format='retina'

#The Deletion Fidelity metric measures how well a saliency-map–based explanation localizes the important features.
#The Insertion Fidelity metric measures how well a saliency-map–based explanation can find elements that are minimal for the predictions.
#MuFidelity is a fidelity metric measuring the correlation between important variables defined by the explanation method and the decline in the model score when these variables are reset to a baseline state.
#Average Stability is a Stability metric measuring how similar are explanations of similar inputs.
import xplique
from xplique.metrics import AverageStability,MuFidelity,Insertion,Deletion
from xplique.plots import plot_attributions
from xplique.attributions import (Saliency, GradientInput, IntegratedGradients, SmoothGrad, VarGrad,
                                  SquareGrad, GradCAM, Occlusion, Rise, GuidedBackprop,
                                  GradCAMPP, Lime, KernelShap, SobolAttributionMethod)

In [ ]:
# Load your TensorFlow model
index = 10
model = get_model()
x_batch, y_batch = next(iter(global_test_gen))
input_image = x_batch[index]
# Add a batch dimension to the first image in the batch
x_single = np.expand_dims(input_image, axis=0)  # This changes the shape from (224, 224, 3) to (1, 224, 224, 3)
y_predicted = model.predict(x_single)
pct = np.max(y_predicted)
predicted_indices = np.argmax(y_predicted, axis=1)
# Map indices to class labels
predicted_labels = [class_labels[idx] for idx in predicted_indices]

print('Predicted_label: '+predicted_labels[0])
print("Prediction Confidence Percentage is: ", pct*100)
print("IntegratedGradients Explainer")
model.layers[-1].activation = tf.keras.activations.linear
explainers = {
    "IntegratedGradients": IntegratedGradients(model,
                                output_layer=-1, batch_size=16,
                                steps=50, baseline_value=0)
}

# iterate on all methods
for method_name, explainer in explainers.items():
    # compute explanation by calling the explainer
    explanation = explainer.explain(x_single, y_predicted)
    plot_attributions(explanation, x_single, img_size=3, cols=1, cmap='jet', alpha=0.3, absolute_value=False,)
    plt.show()

print("Segments")
from lime.wrappers.scikit_image import SegmentationAlgorithm
from skimage.segmentation import mark_boundaries

segmentation_fn = SegmentationAlgorithm(
    'slic',
    kernel_size=4,
    max_dist=200,
    ratio=0.2,
    random_seed=42
)

segments = segmentation_fn(input_image)
plt.imshow(mark_boundaries(np.array(input_image), segments));